In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 


not here


In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [3]:
##### LOAD BOUNDARIES
data_file = 'bi.pickle'
with open(data_file,'rb') as f:
    load_array= pickle.load(f)
exc = load_array[0]
inh = load_array[1]
print(len(exc))

147


In [4]:
t0_array = np.arange(380., -1, -1.)
t_pen = t0_array / 400.

bestControl_1 = [None] * len(t_pen)
bestState_1 = [None] * len(t_pen)
cost_1 = [None] * len(t_pen)
runtime_1 = [None] * len(t_pen)
grad_1 = [None] * len(t_pen)
phi_1 = [None] * len(t_pen)
costnode_1 = [None] * len(t_pen)
weights_1 = [None] * len(t_pen)

In [5]:
initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

cgv_list = [None, "HS", "FR", "PR", "CD", "LS", "DY", "WYL", "HZ", None]

In [6]:
dur_pre = 10
dur_post = 10

i = 20

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-32
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

c_var = [ [0], [1], [0,1]]
p_var = [ [0], [0], [0]]

### CURRENTS
cntrl_vars_0 = [0,1]
prec_vars = [0]

max_I = [-3., 3.]
factor_wp = 1. / 0.05
factor_ws = 0.
factor_we = 1.
dur = 400
trans_time = 0.95
    
maxC = [5., -5., 0.18, 0.]

n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [7]:
init_file = 'control_4.pickle'
final_file_1 = 'control_tradeoff_4a.pickle'

In [8]:
if os.path.isfile(init_file) :
    print("init file found")
    
    with open(init_file,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1[0] = load_array[0][i]
    bestState_1[0] = load_array[1][i]
    cost_1 [0]= load_array[2][i]
    runtime_1[0] = load_array[3][i]
    grad_1 [0]= load_array[4][i]
    phi_1[0] = load_array[5][i]
    costnode_1[0] = load_array[6][i]
    weights_1[0] = load_array[7][i]

init file found
147


In [9]:
if os.path.isfile(final_file_1) :
    print("final file found")
    
    with open(final_file_1,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1 = load_array[0]
    bestState_1 = load_array[1]
    cost_1 = load_array[2]
    runtime_1 = load_array[3]
    grad_1 = load_array[4]
    phi_1 = load_array[5]
    costnode_1 = load_array[6]
    weights_1 = load_array[7]

In [10]:
# get initial parameters and target states

print("------- ", i, exc[i], inh[i])
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.

aln.params.duration = 3000.

control0 = aln.getZeroControl()
control0 = functions.step_control(aln, maxI_ = max_I[0])

aln.run(control=control0)

target_rates = np.zeros((2))
target_rates[0] = aln.rates_exc[0,-1] 
target_rates[1] = aln.rates_inh[0,-1]

control0 = functions.step_control(aln, maxI_ = max_I[1])
aln.run(control=control0)

init_state_vars = np.zeros(( len(state_vars) ))
for j in range(len(state_vars)):
    if aln.state[state_vars[j]].size == 1:
        init_state_vars[j] = aln.state[state_vars[j]][0]
    else:
        init_state_vars[j] = aln.state[state_vars[j]][0,-1]

initVars[i] = init_state_vars

aln.params.duration = dur

target[i] = aln.getZeroTarget()
target[i][:,0,:] = target_rates[0]
target[i][:,1,:] = target_rates[1]

-------  20 0.4500000000000001 0.4750000000000002


In [12]:
# get uncontrolled cost

data.set_parameters(aln)
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.
aln.params.duration = dur

cgv = None

In [11]:
print(np.argmax(np.abs(bestControl_1[0]), axis=2))
print(bestControl_1[0][0,0,1200:2000:40]/5.)

max_percent = (np.argmax(np.abs(bestControl_1[0]), axis=2)[0,0] -100) / 3800
print(max_percent)

[[1516 1493    0    0    0    0]]
[-0.00039327 -0.00099762 -0.00251946 -0.00626378 -0.01508311 -0.03369714
 -0.06457049 -0.09546022 -0.1047776  -0.09275649 -0.07300357 -0.05409275
 -0.03882423 -0.0276243  -0.01928156 -0.01361064 -0.00963111 -0.00697572
 -0.00505355 -0.00354484]
0.3726315789473684


In [13]:
factor_iteration = 1.

##### initial guess
weight_ = 1.
cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

setinit(initVars[i], aln)

weights_1[0] = cost.getParams()

for j in range(1, 381):

    dur = 400. - np.around(j/10., 1)
    T = 4001 - j
    aln.params.duration = dur

    factor_wp = dur / 20.
    cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

    print('-----', j)
    setinit(initVars[i], aln)

    max_it = 3.
    if j > 170:
        max_it = 40.
    elif j > 140:
        max_it = 30.
    elif j > 100.:
        max_it = 20.
    
    if type(bestControl_1[j]) == type(None):
        total_shift = round(j * max_percent * 10)
        prev_shift = round( (j-1) * max_percent * 10)
        shift_ind = total_shift - prev_shift
        #print(total_shift, prev_shift, shift_ind)
        control0 = np.zeros(( bestControl_1[j-1][:,:,100:-101].shape ))
        control0[:,:,:-shift_ind] = 1. * bestControl_1[j-1][:,:,100+shift_ind:-101]
    else:
        if bestControl_1[j].shape[2] == T+200:
            control0 = bestControl_1[j][:,:,100:-100]
        else:
            control0 = bestControl_1[j][:,:,100:-(100+j)]

    bestControl_1[j], bestState_1[j], cost_1[j], runtime_1[j], grad_1[j], phi_1[j], costnode_1[j] = aln.A1(
        control0, target[i], c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = [0,1],
        prec_variables_ = prec_vars, transition_time_ = t_pen[j])

    print(costnode_1[j][0][0][0], costnode_1[j][2][0][1])

    print(costnode_1[0][0][0][0], costnode_1[0][2][0][1])
        
    with open(final_file_1,'wb') as f:
        pickle.dump([bestControl_1, bestState_1, cost_1, runtime_1, grad_1, phi_1,
                costnode_1, weights_1], f)

set cost params:  20.0 1.0 0.0
set cost params:  19.994999999999997 1.0 0.0
----- 1
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.2901369153137408
Gradient descend method:  None
RUN  1 , total integrated cost =  2.290134493512389
RUN  2 , total integrated cost =  2.290134493431805
RUN  3 , total integrated cost =  2.290134493388644
RUN  3 , total integrated cost =  2.290134493388644
Improved over  3  iterations in  31.628300969998236  seconds by  0.00010575459836559276  percent.


ERROR:root:Problem in initial value trasfer


Problem in initial value trasfer:  Vmean_exc -70.82513682339447 -70.8257659079899
3.3047339633896565e-08 0.15737240314787881
1.689941456144599e-07 0.15736824197878707
set cost params:  19.990000000000002 1.0 0.0
----- 2
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.2901345576089955
Gradient descend method:  None
RUN  1 , total integrated cost =  2.2901345560272173
RUN  2 , total integrated cost =  2.2901345559411688


ERROR:root:Problem in initial value trasfer


RUN  3 , total integrated cost =  2.2901345559009845
RUN  3 , total integrated cost =  2.2901345559009845
Improved over  3  iterations in  1.7145258410018869  seconds by  7.458125139692129e-08  percent.
Problem in initial value trasfer:  Vmean_exc -70.82908658658431 -70.82969703523284
3.442262953721786e-08 0.1573726753417196
1.689941456144599e-07 0.15736824197878707
set cost params:  19.985 1.0 0.0
----- 3
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.2901346225685746
Gradient descend method:  None
RUN  1 , total integrated cost =  2.2901346210183458
RUN  2 , total integrated cost =  2.290134620930869


...

RUN  40 , total integrated cost =  2.492251074558565
RUN  40 , total integrated cost =  2.492251074558565
Improved over  40  iterations in  13.670903024001746  seconds by  0.7116086298206312  percent.
Problem in initial value trasfer:  Vmean_exc -70.95859373938694 -70.95859375050073
0.0012229592951056292 0.1680757338759647
1.689941456144599e-07 0.15736824197878707
set cost params:  18.425 1.0 0.0
----- 315
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.5249784694654065
Gradient descend method:  None
RUN  1 , total integrated cost =  2.507941088696681
RUN  2 , total integrated cost =  2.5078838262878826
RUN  3 , total integrated cost =  2.507753704645032
RUN  4 , total integrated cost =  2.50760560155114
RUN  5 , total integrated cost =  2.5075010765877344
RUN  6 , total integrated cost =  2.507402551010656
RUN  7 , total integrated cost =  2.5073767829506823
RUN  8 , total integrated cost =  2.507347867430149
RUN  9 , total integrated cost =  2.50733164229748

ERROR:root:Problem in initial value trasfer


RUN  40 , total integrated cost =  2.5046786111816126
RUN  40 , total integrated cost =  2.5046786111816126
Improved over  40  iterations in  13.513365584003623  seconds by  0.8039616388527833  percent.
Problem in initial value trasfer:  Vmean_exc -70.95859382763153 -70.95859383832982
0.0012930326587643978 0.17000828707555266
1.689941456144599e-07 0.15736824197878707
set cost params:  18.419999999999998 1.0 0.0
----- 316
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.5306668620959787
Gradient descend method:  None
RUN  1 , total integrated cost =  2.5190004332367253
RUN  2 , total integrated cost =  2.518969044477924
RUN  3 , total integrated cost =  2.518920251410945
RUN  4 , total integrated cost =  2.518891292214586
RUN  5 , total integrated cost =  2.5188324982621353
RUN  6 , total integrated cost =  2.518787172607839
RUN  7 , total integrated cost =  2.518727318796604
RUN  8 , total integrated cost =  2.5186777619459235
RUN  9 , total integrated cost =  

ERROR:root:Problem in initial value trasfer


RUN  40 , total integrated cost =  2.517004364024246
RUN  40 , total integrated cost =  2.517004364024246
Improved over  40  iterations in  12.825006181003118  seconds by  0.5398773847466174  percent.
Problem in initial value trasfer:  Vmean_exc -70.95859390365365 -70.958593913994
0.0013621576180822125 0.1698207405818576
1.689941456144599e-07 0.15736824197878707
set cost params:  18.415 1.0 0.0
----- 317
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.5418935933573374
Gradient descend method:  None
RUN  1 , total integrated cost =  2.5320840866569694
RUN  2 , total integrated cost =  2.5320484462535466
RUN  3 , total integrated cost =  2.532009782459257
RUN  4 , total integrated cost =  2.531973990126176
RUN  5 , total integrated cost =  2.531905259361156
RUN  6 , total integrated cost =  2.5318370810132085
RUN  7 , total integrated cost =  2.5317675331387397
RUN  8 , total integrated cost =  2.531698848893207
RUN  9 , total integrated cost =  2.53136193484611